In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

## List Assign Variables

In [2]:
arg_date = '2021-10-18'
src_format = '%Y-%m-%d'
src_bucket = 'deutsche-boerse-xetra-pds'
trg_bucket = 'xetra-proj01'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [3]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

## Connect AWS S3 bucket via AWS boto3

##### Then for loop through the s3 bucket -> key.split('/') (key='2021-10-18/2021-10-18_BINS_XETR21.csv') use 2021-10-18 >= arg_date_dt

In [4]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]

#### Function CSV File

In [5]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [6]:
df_all = df_all.loc[:, columns]

In [7]:
df_all.dropna(inplace=True)

In [8]:
df_all.head()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2021-10-18,07:00,22.16,22.32,22.02,22.32,3881
1,DE000A0DJ6J9,2021-10-18,07:00,40.80,40.80,40.56,40.70,5728
2,DE000A0D6554,2021-10-18,07:00,15.20,15.22,15.04,15.05,43682
3,DE000A0D9PT0,2021-10-18,07:00,189.10,189.10,187.75,188.10,2635
4,DE000A0HN5C6,2021-10-18,07:00,52.94,52.98,52.94,52.98,30419


## Data Analysis

In [9]:
df_all.describe()

,StartPrice,MaxPrice,MinPrice,EndPrice
count,296727.000000,296727.000000,296727.000000,296727.000000
mean,98.642528,98.658715,98.625820,98.641831
std,689.411545,689.417812,689.404607,689.410775
min,0.014500,0.014500,0.012000,0.012000
25%,17.974000,17.980000,17.964000,17.970000
50%,46.555000,46.575000,46.550000,46.560000
75%,97.500000,97.528000,97.490000,97.500000
max,119148.000000,119148.000000,119148.000000,119148.000000


In [10]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296727 entries, 0 to 296726
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ISIN          296727 non-null  object 
 1   Date          296727 non-null  object 
 2   Time          296727 non-null  object 
 3   StartPrice    296727 non-null  float64
 4   MaxPrice      296727 non-null  float64
 5   MinPrice      296727 non-null  float64
 6   EndPrice      296727 non-null  float64
 7   TradedVolume  296727 non-null  object 
dtypes: float64(4), object(4)
memory usage: 20.4+ MB


## Get opening price per ISIN and day

In [11]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [12]:
df_all.head()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2021-10-18,07:00,22.16,22.32,22.02,22.32,3881,22.16
1,DE000A0DJ6J9,2021-10-18,07:00,40.80,40.80,40.56,40.70,5728,40.80
2,DE000A0D6554,2021-10-18,07:00,15.20,15.22,15.04,15.05,43682,15.20
3,DE000A0D9PT0,2021-10-18,07:00,189.10,189.10,187.75,188.10,2635,189.10
4,DE000A0HN5C6,2021-10-18,07:00,52.94,52.98,52.94,52.98,30419,52.94


## Get closing price per ISIN and day

In [13]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [14]:
df_all.head()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2021-10-18,07:00,22.16,22.32,22.02,22.32,3881,22.16,22.40
1,DE000A0DJ6J9,2021-10-18,07:00,40.80,40.80,40.56,40.70,5728,40.80,40.24
2,DE000A0D6554,2021-10-18,07:00,15.20,15.22,15.04,15.05,43682,15.20,14.99
3,DE000A0D9PT0,2021-10-18,07:00,189.10,189.10,187.75,188.10,2635,189.10,187.30
4,DE000A0HN5C6,2021-10-18,07:00,52.94,52.98,52.94,52.98,30419,52.94,52.98


## Aggregations

In [15]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=True).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [16]:
df_all.head()

opening_price_eur  closing_price_eur  \
ISIN         Date                                               
AT00000FACC2 2021-10-18               9.19               9.10   
             2021-10-19               8.99               9.08   
             2021-10-20               9.14               9.03   
AT0000606306 2021-10-18              24.76              24.58   
             2021-10-19              24.50              24.54   

                         minimum_price_eur  maximum_price_eur  \
ISIN         Date                                               
AT00000FACC2 2021-10-18               9.10               9.19   
             2021-10-19               8.99               9.08   
             2021-10-20               9.03               9.14   
AT0000606306 2021-10-18              24.58              24.76   
             2021-10-19              24.40              24.76   

                         daily_traded_volume  
ISIN         Date                             
AT00000FACC2 2021-10-18                  118  
             2021-10-19                    1  
             2021-10-20                   28  
AT0000606306 2021-10-18                   50  
             2021-10-19                 2053

In [17]:
df_all.reset_index(inplace=True)

In [18]:
df_all.head()

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT00000FACC2,2021-10-18,9.19,9.10,9.10,9.19,118
1,AT00000FACC2,2021-10-19,8.99,9.08,8.99,9.08,1
2,AT00000FACC2,2021-10-20,9.14,9.03,9.03,9.14,28
3,AT0000606306,2021-10-18,24.76,24.58,24.58,24.76,50
4,AT0000606306,2021-10-19,24.50,24.54,24.40,24.76,2053


## Percent Change Prev Closing

In [19]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

## Quick and Dirty Project Report

In [20]:
df_all.head()

In [24]:
df_all.memory_usage()

Index                    128
ISIN                   74016
Date                   74016
opening_price_eur      74016
closing_price_eur      74016
minimum_price_eur      74016
maximum_price_eur      74016
daily_traded_volume    74016
prev_closing_price     74016
dtype: int64